In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [2]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

In [3]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [4]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [5]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [6]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [7]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [8]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [9]:
unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [10]:
embedding_dimension = 32

In [11]:
user_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None
    ),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [12]:
movie_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None
    ),
    tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [13]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates=movies.batch(128).map(movie_model)
)

In [14]:
task = tfrs.tasks.Retrieval(
    metrics=metrics
)

In [15]:
class MovielensModel(tfrs.Model):
    def __init__(self, user_model, movie_model):
        super().__init__()
        self.movie_model: tf.keras.Model = movie_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        positive_movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, positive_movie_embeddings)

In [16]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [17]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [18]:
model.fit(cached_train, epochs=10)

Epoch 1/10
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


10/10 [==============================] - 4s 253ms/step - factorized_top_k/top_1_categorical_accuracy: 7.5000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0045 - factorized_top_k/top_50_categorical_accuracy: 0.0461 - factorized_top_k/top_100_categorical_accuracy: 0.1095 - loss: 69885.1101 - regularization_loss: 0.0000e+00 - total_loss: 69885.1101

In [19]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 139ms/step - factorized_top_k/top_1_categorical_accuracy: 2.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0037 - factorized_top_k/top_10_categorical_accuracy: 0.0100 - factorized_top_k/top_50_categorical_accuracy: 0.0841 - factorized_top_k/top_100_categorical_accuracy: 0.1813 - loss: 31678.6315 - regularization_loss: 0.0000e+00 - total_loss: 31678.6315

{'factorized_top_k/top_1_categorical_accuracy': 0.00019999999494757503,
 'factorized_top_k/top_5_categorical_accuracy': 0.003650000086054206,
 'factorized_top_k/top_10_categorical_accuracy': 0.010049999691545963,
 'factorized_top_k/top_50_categorical_accuracy': 0.08410000056028366,
 'factorized_top_k/top_100_categorical_accuracy': 0.18129999935626984,
 'loss': 28730.44140625,
 'regularization_loss': 0,
 'total_loss': 28730.44140625}

In [20]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

In [21]:
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)' b'Rudy (1993)'
 b'Aristocats, The (1970)']


In [22]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.user_model)
scann_index.index(movies.batch(100).map(model.movie_model), movies)

In [23]:
_, titles = scann_index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)' b'Affair to Remember, An (1957)'
 b'Rent-a-Kid (1995)']


In [24]:
export_path = './scann_recommend/1/'
tf.keras.models.save_model(
    scann_index,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"])
)
print("Saved model")

INFO:tensorflow:Assets written to: ./scann_recommend/1/assets


INFO:tensorflow:Assets written to: ./scann_recommend/1/assets


Saved model


In [25]:
for dict_batch in cached_test.take(1):
    print(dict_batch['movie_title'], dict_batch['user_id'])

tf.Tensor(
[b'M*A*S*H (1970)' b'Volcano (1997)' b'2001: A Space Odyssey (1968)' ...
 b'Nosferatu a Venezia (1986)' b'African Queen, The (1951)'
 b'Stargate (1994)'], shape=(4096,), dtype=string) tf.Tensor([b'346' b'602' b'393' ... b'405' b'14' b'303'], shape=(4096,), dtype=string)


In [26]:
user_id_np = [user.decode("utf-8") for user in dict_batch['user_id'][0:3].numpy()]

In [27]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": user_id_np})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ... erving_default", "instances": ["346", "602", "393"]}


In [28]:
import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/scann_recommend:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

In [29]:
predictions

[{'output_2': ['Marked for Death (1990)',
   'Assassins (1995)',
   'Hard Target (1993)',
   'Young Guns II (1990)',
   'Blown Away (1994)',
   'Sliver (1993)',
   'Under Siege 2: Dark Territory (1995)',
   'Desperado (1995)',
   'Jade (1995)',
   'Dead Presidents (1995)'],
  'output_1': [3.41740823,
   3.1108222,
   3.0585289,
   3.02264071,
   3.02110243,
   3.00913978,
   2.93292093,
   2.77740645,
   2.75194287,
   2.6203537]},
 {'output_2': ['Jungle2Jungle (1997)',
   'George of the Jungle (1997)',
   'Beautician and the Beast, The (1997)',
   'Saint, The (1997)',
   'Air Bud (1997)',
   'Volcano (1997)',
   'Fire Down Below (1997)',
   'Booty Call (1997)',
   'Scream 2 (1997)',
   'How to Be a Player (1997)'],
  'output_1': [5.3835,
   4.90118313,
   4.86500931,
   4.83494091,
   4.82883549,
   4.47923231,
   4.44901085,
   4.43092394,
   4.42662716,
   4.39045334]},
 {'output_2': ['Low Down Dirty Shame, A (1994)',
   'Lightning Jack (1994)',
   'Steal Big, Steal Little (1995)',
